This notebook replicates the final permutation robustness analysis for hook errors from the distance 3 study, now for distance 5. It programmatically derives stabilizer connectivity, targets only weight-4 stabilizers, and injects a Pauli error (hook) on the ancilla after a specified CNOT (hook step) within each stabilizer's 4 CNOT sequence.

Adjust HOOK_STEP (0..3) to choose after which CNOT the ancilla error is injected (0 means after 1st, 1 means after 2nd, etc.). The same ordering permutation is applied to all weight-4 stabilizers. For each permutation of [0,1,2,3], we inject a single hook error on every weight-4 stabilizer (one at a time), propagate, and accumulate whether any logical X or Z line meets the majority-error threshold (>=3 along a 5-qubit line).

Output: Robust permutations (those producing zero logical row/column errors across all individual weight-4 stabilizer hook injections) plus full results.

In [6]:
from error_prop_tools import (
    generate_rotated_surface_code,
    parse_qubit_map,
    find_neighboring_data_qubits,
    generate_surface_code_qasm,
)
import itertools
import re

# --- Parameters ---
D = 5
HOOK_STEP = 1   # 0-based: 0..3 (after which CNOT in the 4-CNOT sequence we inject the ancilla error)
ERROR_TYPE_BY_STAB_TYPE = {'X': 'X', 'Z': 'Z'}  # hook Pauli placed on ancilla depending on stabilizer type

# --- Build connectivity programmatically ---
sc_map = generate_rotated_surface_code(D)
_, ancillas = parse_qubit_map(sc_map)  # data qubits not directly needed here

# Determine neighbor data qubits for each ancilla and classify weight
connectivity = {}
ancilla_types = {}
for anc in ancillas:
    label = anc['label']
    r_m, c_m = anc['matrix_coord']
    neighbors = find_neighboring_data_qubits(sc_map, r_m, c_m)
    connectivity[label] = neighbors
    ancilla_types[label] = label[0]

# Filter to weight-4 stabilizers only (internal checks)
weight4_stabs = sorted([lab for lab, neigh in connectivity.items() if len(neigh) == 4], key=lambda s: (s[0], int(s[1:])))
print(f"Found {len(weight4_stabs)} weight-4 stabilizers (internal):")
print(weight4_stabs)

# --- Logical line definitions for rotated surface code (distance 5) ---
D2 = D * D
Z_LINES = [list(range(r*D, r*D + D)) for r in range(D)]          # horizontal rows (data indices)
X_LINES = [list(range(c, D2, D)) for c in range(D)]              # vertical columns
EXPECTED_DATA = D2
print(f"Expected data qubits = {EXPECTED_DATA}")

# --- QASM schedule parsing utilities ---

def qasm_to_schedule(qasm_str):
    lines = [ln.strip() for ln in qasm_str.splitlines()]
    m_q = next((re.match(r"qreg q\[(\d+)\];", ln) for ln in lines if ln.startswith('qreg q[')), None)
    m_a = next((re.match(r"qreg a\[(\d+)\];", ln) for ln in lines if ln.startswith('qreg a[')), None)
    num_data = int(m_q.group(1)) if m_q else 0
    num_anc = int(m_a.group(1)) if m_a else 0
    if num_data != EXPECTED_DATA:
        print(f"WARNING: QASM header reports num_data={num_data} (expected {EXPECTED_DATA})")

    anc_label_to_aidx = {}
    per_anc = {}
    order = []
    cur = None
    for ln in lines:
        if ln.startswith('// Stabilizer for '):
            m = re.match(r"// Stabilizer for\s+([XZ]\d+)\s+\(a\[(\d+)\]\)", ln)
            if not m: continue
            cur = m.group(1)
            aidx = int(m.group(2))
            anc_label_to_aidx[cur] = aidx
            per_anc[cur] = {'H_pre': False, 'cnots': [], 'H_post': False}
            order.append(cur)
        elif ln.startswith('h a['):
            if cur is None: continue
            if not per_anc[cur]['H_pre']:
                per_anc[cur]['H_pre'] = True
            else:
                per_anc[cur]['H_post'] = True
        elif ln.startswith('cx '):
            if cur is None: continue
            m1 = re.match(r"cx\s+q\[(\d+)\],\s*a\[(\d+)\];", ln)
            m2 = re.match(r"cx\s+a\[(\d+)\],\s*q\[(\d+)\];", ln)
            if m1:
                dq = int(m1.group(1))
                per_anc[cur]['cnots'].append(('D->A', dq))
            elif m2:
                dq = int(m2.group(2))
                per_anc[cur]['cnots'].append(('A->D', dq))
    return num_data, num_anc, anc_label_to_aidx, per_anc, order

# --- Pauli helpers ---

def p_encode(p):
    return {'I': (0,0), 'X': (1,0), 'Z': (0,1), 'Y': (1,1)}[p]

def p_decode(xz):
    x,z = xz
    if (x,z)==(0,0): return 'I'
    if (x,z)==(1,0): return 'X'
    if (x,z)==(0,1): return 'Z'
    return 'Y'

# --- Hook simulation ---

def simulate_hook_paulis_qasm_d5(ordering, target_stab, hook_step, error_type):
    perm4 = list(ordering)
    custom_orders = {lab: perm4 for lab in weight4_stabs}
    qasm = generate_surface_code_qasm(D, custom_cnot_orderings=custom_orders)
    num_data, num_anc, anc_label_to_aidx, per_anc, anc_order = qasm_to_schedule(qasm)
    paulis = {q: (0,0) for q in range(num_data + num_anc)}

    def H_on(q):
        x,z = paulis[q]; paulis[q] = (z,x)
    def CNOT_(c,t):
        xc,zc = paulis[c]; xt,zt = paulis[t]
        xt ^= xc; zc ^= zt
        paulis[c] = (xc,zc); paulis[t] = (xt,zt)
    def add_pauli(q, p):
        ax,az = paulis[q]; bx,bz = p_encode(p)
        paulis[q] = (ax^bx, az^bz)

    for lab in anc_order:
        steps = per_anc[lab]
        aidx = anc_label_to_aidx[lab]
        aq = num_data + aidx
        if steps['H_pre']:
            H_on(aq)
        for i,(kind,dq) in enumerate(steps['cnots']):
            if kind=='D->A':
                CNOT_(dq, aq)
            else:
                CNOT_(aq, dq)
            if lab==target_stab and i==hook_step:
                add_pauli(aq, error_type)
        if steps['H_post']:
            H_on(aq)

    data_paulis = [p_decode(paulis[q]) for q in range(num_data)]
    return data_paulis

# --- Line weight metrics ---

def line_weights(data_paulis):
    z_line_w = [sum(1 for q in row if data_paulis[q] in ('Z','Y')) for row in Z_LINES]
    x_line_w = [sum(1 for q in col if data_paulis[q] in ('X','Y')) for col in X_LINES]
    return z_line_w, x_line_w

# --- Permutation analysis: record maximum physical error weight on any line ---
perms = list(itertools.permutations([0,1,2,3]))
results = {}
print(f"Evaluating {len(perms)} permutations (metric: max line weights) HOOK_STEP={HOOK_STEP}")
for p in perms:
    max_z = 0; max_x = 0
    # consider worst-case single-hook among weight-4 stabilizers
    for stab in weight4_stabs:
        stab_type = ancilla_types[stab]
        err_type = ERROR_TYPE_BY_STAB_TYPE[stab_type]
        data_ps = simulate_hook_paulis_qasm_d5(p, stab, hook_step=HOOK_STEP, error_type=err_type)
        z_w, x_w = line_weights(data_ps)
        max_z = max(max_z, max(z_w))
        max_x = max(max_x, max(x_w))
    results[p] = (max_z, max_x)

# Define "robust" now as keeping max line weight <= 2 (no distance reduction below 3 for d=5) in both X and Z
robust = sorted([p for p,(mz,mx) in results.items() if mz <= 1 and mx <= 1])
print("\nRobust permutations (max line weight <=2 in both bases for all single hooks):")
print(robust)
print(f"Count robust: {len(robust)} / 24")

print("\nFull results (permutation -> (maxZlineWeight, maxXlineWeight)):")
for p,res in sorted(results.items()):
    print(p, res)

Found 16 weight-4 stabilizers (internal):
['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'Z0', 'Z1', 'Z4', 'Z5', 'Z6', 'Z7', 'Z10', 'Z11']
Expected data qubits = 25
Evaluating 24 permutations (metric: max line weights) HOOK_STEP=1

Robust permutations (max line weight <=2 in both bases for all single hooks):
[(0, 3, 1, 2), (0, 3, 2, 1), (1, 2, 0, 3), (1, 2, 3, 0), (2, 1, 0, 3), (2, 1, 3, 0), (3, 0, 1, 2), (3, 0, 2, 1)]
Count robust: 8 / 24

Full results (permutation -> (maxZlineWeight, maxXlineWeight)):
(0, 1, 2, 3) (2, 1)
(0, 1, 3, 2) (2, 1)
(0, 2, 1, 3) (1, 2)
(0, 2, 3, 1) (1, 2)
(0, 3, 1, 2) (1, 1)
(0, 3, 2, 1) (1, 1)
(1, 0, 2, 3) (2, 1)
(1, 0, 3, 2) (2, 1)
(1, 2, 0, 3) (1, 1)
(1, 2, 3, 0) (1, 1)
(1, 3, 0, 2) (1, 2)
(1, 3, 2, 0) (1, 2)
(2, 0, 1, 3) (1, 2)
(2, 0, 3, 1) (1, 2)
(2, 1, 0, 3) (1, 1)
(2, 1, 3, 0) (1, 1)
(2, 3, 0, 1) (2, 1)
(2, 3, 1, 0) (2, 1)
(3, 0, 1, 2) (1, 1)
(3, 0, 2, 1) (1, 1)
(3, 1, 0, 2) (1, 2)
(3, 1, 2, 0) (1, 2)
(3, 2, 0, 1) (2, 1)
(3, 2, 1, 0) (2, 1)

Robust p